In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
print(os.listdir("../input"))

# Read input files
destinations = pd.read_csv("../input/destinations.csv")
# test = pd.read_csv("../input/test.csv")
# train = pd.read_csv("../input/train.csv")

# Read and split train data into train_new and test_new
train_new = pd.DataFrame(columns=["date_time", "site_name", "posa_continent", "user_location_country", "user_location_region", "user_location_city", "orig_destination_distance", "user_id", "is_mobile", "is_package", "channel", "srch_ci", "srch_co", "srch_adults_cnt", "srch_children_cnt", "srch_rm_cnt", "srch_destination_id", "srch_destination_type_id", "hotel_continent", "hotel_country", "hotel_market", "is_booking", "cnt", "hotel_cluster"])
test_new = pd.DataFrame(columns=["date_time", "site_name", "posa_continent", "user_location_country", "user_location_region", "user_location_city", "orig_destination_distance", "user_id", "is_mobile", "is_package", "channel", "srch_ci", "srch_co", "srch_adults_cnt", "srch_children_cnt", "srch_rm_cnt", "srch_destination_id", "srch_destination_type_id", "hotel_continent", "hotel_country", "hotel_market", "is_booking", "cnt", "hotel_cluster"])
chunksize = 10 ** 6
for chunk in pd.read_csv("../input/train.csv", chunksize=chunksize):
    chunk["date_time"] = pd.to_datetime(chunk["date_time"])
    chunk["year"] = chunk["date_time"].dt.year
    chunk["month"] = chunk["date_time"].dt.month
    
    # In the original train and test DataFrames, test contained data from 2015, and train contained data from 2013 and 2014. 
    # We split this data so that anything Jan - Aug 2013 is in train_new, and the rest of 2013 is in test_new. 
    # This gives us smaller training and testing sets with similar characteristics to train and test.
    train_new = train_new.append(chunk[((chunk.year == 2013) & (chunk.month <= 8))], ignore_index=True)
    test_new = test_new.append(chunk[((chunk.year == 2013) & (chunk.month > 8))], ignore_index=True)

print(train_new.shape) # train_new.shape = (7624260, 26)
print(test_new.shape) # test_new.shape =  (3562621, 26)
print("Done loading data")

In [ ]:
import random

# Pick 200000 users from train_new set and pick coresponding set of users from test_new
# That will further downsample our dataset, and keeps the characteristic of the original data set
# which is that all the test user ids can be found in the train
unique_users = set(train_new.user_id.unique())

sel_user_ids = random.sample(unique_users, 300000)
train_new_1 = train_new[train_new.user_id.isin(sel_user_ids)]
test_new_1 = test_new[test_new.user_id.isin(sel_user_ids)]

# Test contains only booking events, so we'll need to sample t2 to only contain bookings as well
test_new_1 = test_new_1[test_new_1.is_booking == True]

print(train_new_1.shape)
print(test_new_1.shape)

train_new_1.to_csv('train_new.csv', sep=',')
test_new_1.to_csv('test_new.csv', sep=',')


In [ ]:
# Start your code from here. No need to rerun the first two cells of code
# The downsample data is already saved in file train_new.csv and test_new.csv
